In [14]:
from PIL import Image, ImageDraw
from sympy import *
'''
x, y = symbols('x y')
gfg_exp = x**2-4
  
print("Before Integration : {}".format(gfg_exp))
  
# Use sympy.integrate() method
intr = solve(gfg_exp, x)
  
print("After Integration : {}".format(intr))
'''

Before Integration : x**2 - 4
After Integration : [-2, 2]


In [10]:
def transform(x1,y1,x2,y2,x3,y3,x11,y11,x22,y22,x33,y33):
    
    a = Symbol('a')
    b = Symbol('b')
    c = Symbol('c')
    d = Symbol('d')
    e = Symbol('e')
    f = Symbol('f')
    
    fuc1 = x11*a + y11*b + c - x1   ##  解 x = x'a+ y'b+ c 此func 解出 a b c   
    fuc2 = x22*a + y22*b + c - x2
    fuc3 = x33*a + y33*b + c - x3
    
    sol1 = solve((fuc1,fuc2,fuc3), a,b,c)
    
    fuc4 = x11*d + y11*e + f - y1   ##  解 y = x'd+ y'e+ f 此func 解出 d e f
    fuc5 = x22*d + y22*e + f - y2
    fuc6 = x33*d + y33*e + f - y3
    
    sol2 = solve((fuc4,fuc5,fuc6), d,e,f)
    
    return sol1[a], sol1[b], sol1[c], sol2[d], sol2[e], sol2[f]

In [11]:
def bilinear(x,y,image):
    
    l = int(x)
    k = int(y)
    a = x - l     # 原始x - int(x) 得出a比例
    b = y - k     # 原始y - int(y) 得出b比例
    
    R = (1-a) * (1-b) * image.getpixel((l,k))[0] + a * (1-b) * image.getpixel((l+1,k))[0] + \
        (1-a) * b * image.getpixel((l,k+1))[0] + a * b * image.getpixel((l+1,k+1))[0]
    
    G = (1-a) * (1-b) * image.getpixel((l,k))[1] + a * (1-b) * image.getpixel((l+1,k))[1]+ \
        (1-a) * b * image.getpixel((l,k+1))[1] + a * b * image.getpixel((l+1,k+1))[1]
    
    B = (1-a) * (1-b) * image.getpixel((l,k))[2] + a * (1-b) * image.getpixel((l+1,k))[2]+ \
        (1-a) * b * image.getpixel((l,k+1))[2] + a * b * image.getpixel((l+1,k+1))[2]
    
    return R,G,B

In [12]:
def main(img1, img2, x1, y1, x2, y2, x3, y3, x11, y11, x22, y22, x33, y33, output_img):
    image1 = Image.open(img1) # Need to rotate image
    image2 = Image.open(img2) # Original
    w1, h1 = image1.size
    w2, h2 = image2.size
    
    a, b, c, d, e, f = transform(x1,y1,x2,y2,x3,y3,x11,y11,x22,y22,x33,y33)
    
    ground_img = Image.new("RGB", (w2 + int(w2*0.7), int(h2*1.1)), (255,255,255)) # 建立背景大白圖
    draw = ImageDraw.Draw(ground_img)
    
    new_x_range = w2 + int(w2*0.7)-1
    for new_i in range(0, w2+int(w2*0.7)-1):

        for new_j in range(0, int(h2*1.1)-1):
            
            if 0 <= new_i <= w2-1 and 0 <= new_j <= h2-1:
                draw.point((new_i, new_j), fill=(image2.getpixel((new_i,new_j))))
            
            else:
                x = (a * new_i) + (b * new_j) + c
                y = (d * new_i) + (e * new_j) + f
                
                if 0 <= x <= w1-1 and 0 <= y <= h1-1:
                    R,G,B = bilinear(x,y,image1)
                    draw.point((new_i, new_j), fill=(int(R),int(G),int(B)))
                else:
                    continue
    ground_img.save(output_img)
    
main('03.jpg','01.jpg', x1=184, y1=378, x2=344, y2=675, x3=393, y3=435, x11=704, y11=330, x22=787, \
     y22=658, x33=902, y33=437, output_img='output_img_combine2.jpg')